In [2]:
# Install required packages
import sys
import os
import time
import subprocess
!{sys.executable} -m pip install reportlab svglib PyPDF2 img2pdf pillow

# Load packages
from xml.dom import minidom
from reportlab.graphics import renderPDF, renderPM
from svglib.svglib import svg2rlg
from pathlib import Path
from PyPDF2 import PdfFileReader, PdfFileWriter
from PIL import Image
import img2pdf

In [15]:
class main_process:
    def __init__(self, color, output_path, quality=5):
        self.color = color
        self.output_path = output_path
        self.original_size = 0
        self.final_size = 0
        self.quality = quality # 0 to 100
    
    def edit_and_save(self, file, index):    
        # Get XML doc
        mydoc = minidom.parse(file)

        # Get layers
        items = mydoc.getElementsByTagName('g')

        # Update stroke attribute color in each layer of the doc
        for elem in items:
            path = elem.getElementsByTagName('path')
            path[0].setAttribute('stroke', self.color)

        # Save XML file
        file_handle = open("new.svg","w")
        file_handle.write(mydoc.toxml())
        drawing = svg2rlg("new.svg")
        
        # Save and reduce PNG
        renderPM.drawToFile(drawing, "new.png", "PNG")
        
        # Apply compression
        self.reduce_size()

        # Save PDF file
        file_handle = open(self.output_path + "/" + str(index) + ".pdf", "wb")
        file_handle.write(img2pdf.convert("resized.jpg"))
        # renderPDF.drawToFile(drawing, self.output_path + "/" + str(index) + ".pdf")
    
    def reduce_size(self):
        self.original_size += os.stat('new.png').st_size
        picture = Image.open('new.png')
        dim = picture.size
        picture.save("resized.jpg","JPEG",optimize=True,quality=self.quality) 
        self.final_size += os.stat(os.path.join(os.getcwd(),"resized.jpg")).st_size
            
    def calc_compression(self):
        percent = (self.original_size-self.final_size)/float(self.original_size)*100
        print("File compressed from " + str(self.original_size) + " to " + str(self.final_size) + " or " + str(percent) + " %")
             

In [4]:
class measure_time:
    def __init__(self):
        if sys.platform == 'win32':
            # On Windows, the best timer is time.clock
            self.default_timer = time.clock
        else:
            # On most other platforms the best timer is time.time
            self.default_timer = time.time
    
    def start(self):
        self.ini = self.default_timer()
    
    def end(self, pages):
        print("Editor processed " + str(pages) + " pages in " + str(self.default_timer() - self.ini) + " seconds" )
        

In [5]:
def get_index(x, start_char, end_char):    
    ini = x.rfind(start_char)
    end = x.rfind(end_char)
    y = x[ini+1:end]
    return y

In [6]:
def pdf_cat(output_path, input_paths):
    pdf_writer = PdfFileWriter()
 
    for path in input_paths:
        pdf_reader = PdfFileReader(path)
        for index in range(pdf_reader.getNumPages()):
            page = pdf_reader.getPage(index)
            pdf_writer.addPage(page)
 
    with open(output_path, 'wb') as fh:
        pdf_writer.write(fh)

In [7]:
def sort_paths(x, count):
    l = [None] * count
    for path in x:
        n = get_index(str(path), "/", ".")
        if n.isdigit():
            l[int(n)-1] = str(path)
        else:
            print("-- Warning: " + n + " is not valid index")
    return l

In [17]:
# Configuration defaults (Change according to your needs)
stroke_color = "#FF0000" # default red
inputs_path = "test_files"
outputs_path = "results"
output_pdf_name = "final_result.pdf"
quality = 5 # 5 is the default value

# Create timer object to measure elapsed time
timer = measure_time()

# Main process
timer.start()
print("Editing Remarkable output started!")

pathlist = Path(inputs_path).glob('**/*.svg')
num_of_pages = 0
update_inputs = main_process(stroke_color, outputs_path, quality)
for path in pathlist:
    index = get_index(str(path), " ", ".")
    if index.isdigit():
        num_of_pages = num_of_pages + 1
        update_inputs.edit_and_save(str(path), index)
    else:
        print("-- Warning: " + index + " is not valid index")

print(str(num_of_pages) + " PDFs generated, now compiling final PDF")
paths = Path(outputs_path).glob('**/*.pdf')
pdf_cat(output_pdf_name, sort_paths(paths, num_of_pages))
timer.end(num_of_pages)
update_inputs.calc_compression()
print("Done!")

Editing Remarkable output started!
-- Warning: 6-checkpoint is not valid index
-- Warning: 1-checkpoint is not valid index
-- Warning: 10-checkpoint is not valid index
-- Warning: 100-checkpoint is not valid index
376 PDFs generated, now compiling final PDF
-- Warning: 5-checkpoint is not valid index
-- Warning: 6-checkpoint is not valid index
-- Warning: 1-checkpoint is not valid index
Editor processed 376 pages in 115.65151381492615 seconds
File compressed from 121298151 to 20128495 or 83.40576930970695 %
Done!
